# Tool use

什么是Tool use（工具使用）?

Tool use工具使用,也称为Function call函数调用,是指通过定义和调用外部工具或函数来扩展Claude能力的一种方式。我们可以给Claude提供一组预定义的工具,让它在需要时调用。

你可以把它想象成你**需要先自己构造好的一个工具箱**,里面装满了各种各样的工具函数,每一个工具函数都有自己的特点和用途,**并可以实际运行**。

当你需要 Claude 来帮助你完成某个任务时,你就可以把工具箱交给 Claude 使用。比如说,你想要 Claude 帮你查询此时此刻的某支股票价钱与最近走势,你可以开放"股票查询"的工具函数的权限给Claude,告诉 Claude 这个工具的名称、功能描述和输入参数要求——通过`tools`参数。然后 Claude 会自主通过用户的prompt来判断是否使用使用这个工具来帮你完成任务——可以通过`tool_choice`参数控制是否强制使用工具以及使用那个工具，但需注意，所有的工具函数是我们额外自己写好的，可以实际运行的，已经存在的，我们只是相当于开放这个函数的权限给claude，并通过详细的描述让Claude知道这个函数是做什么的以及应该传入什么样的参数，这样它会从prompt识别评估需不需要调用函数，需要调用哪个函数，以及入参都是什么。

| 类别                                      | 参数名称 | 类型   | 描述                                                                 |
|-----------------------------------------|----------|--------|--------------------------------------------------------------------|
| **必需参数**                              |          |        |                                                                    |
|                                         | model    | string | 指定要使用的 Claude 模型版本。                                     |
|                                         | messages  | array  | 输入对话历史和当前用户输入的消息列表，每个消息包括角色和内容。      |
|                                         | max_tokens | integer | 回答停止前的最大令牌数。模型可能会在达到此最大值之前停止。         |
| **可选参数**                              |          |        |                                                                    |
| 生成控制                                  | temperature | float | 注入响应的随机性的程度。默认为 1.0，范围从 0.0 到 1.0。           |
|                                         | top_k    | integer | 仅从每个后续令牌的前 K 个选项中抽样。用于删除低概率响应的"长尾"。   |
|                                         | top_p    | float  | 使用核心抽样。在核心抽样中，计算所有选项的累积分布，并在达到特定概率后切断。 |
| 系统提示                                 | system | string | 系统提示。用于提供上下文和指令。                                 |
| 停止序列                                  | stop_sequences | array | 自定义文本序列，会使模型停止生成。如果模型遇到这些序列之一，响应将提前结束。 |
| **工具调用**                                  | tool_choice | string | 模型如何使用提供的工具。可以选择特定工具、任何可用工具，或由模型自行决定。 |
|                                         | tools    | array  | 定义模型可能使用的工具。每个工具定义都包括名称和输入模式。         |
| 流式传输参数                                | stream   | boolean | 是否使用服务器发送事件增量流式传输响应。                           |
| 其他参数                                  | metadata | object | 描述请求的元数据。                                                  |
|                                         | anthropic-beta | string | 指定你想使用的 beta 版本。多个版本使用逗号分隔，不带空格。         |
|                                         | anthropic-version | string | 指定使用的 Anthropic API 的版本。                                  |
|                                         | x-api-key | string | 用于认证的独特 API 密钥。通过控制台获取。                           |



工具箱里的每个工具假如开发者想要传给LLM实际使用，要准备好两样东西：
1. 一个Json Schema对象来描述函数名称、函数是用来做什么的函数说明和子集JSon Schema（函数及所需的参数的JSON架构）。名称要符合一定的规则,这样 Claude 才能识别和使用它。函数说明要非常详细,解释工具的具体功能、使用场景和注意事项以及入参说明,这样 Claude 就能更好地理解工具的用途。**优先考虑描述而不是示例。虽然可以在描述中包含使用示例,但这不如全面解释工具的目的和参数重要。**
2. 实际的可以跑通的本身的函数（可选，如果只想得到大模型的回答这部分不用一定要准备好）

有时候一个工具可能还不够,需要多个工具配合使用。对于需要按顺序调用多个工具的任务,Claude会一次调用一个工具。
需要注意的是,使用工具箱里的工具也是要付费的,费用取定价与其他Claude API请求相同,基于发送给模型的总输入token(包括tools参数中的内容)和生成的输出token。所以放入工具参数的描述时,要权衡任务的需求和成本。  

简而言之，它是一种给Claude安装工具箱（外部函数）并赋予它自主判断或可以强制控制是否调用的增强Claude功能的方法，从而可以：

- 使助手能够获取数据：当用户询问“我最近的订单是什么”时，AI 助手需要从内部系统获取最新的客户数据，然后才能对用户生成响应。

- 使助理能够采取行动：AI 助理需要根据用户偏好和日历可用性安排会议。

- 使助手能够执行计算：数学导师助手需要执行数学计算。

- 构建丰富的工作流：一个数据提取管道，用于获取原始文本，然后将其转换为结构化数据（Json）并将其保存在数据库中。

- 修改应用程序的 UI：您可以使用函数调用来根据用户输入更新 UI，例如，在地图上呈现图钉。



## tools

让我们来关注Tool use实际上是如何工作的。首先最重要的是要理解,Claude并不会自己运行任何代码。我们告诉Claude它可以要求我们调用的一组工具,然后我们的工作是实际运行底层工具代码并告诉Claude结果。

请注意,Claude没有访问任何内置的服务器端工具的权限。所有工具必须由你(用户)在每个API请求中明确提供。这意味着你定义可用的工具,包括清晰的描述和输入模式,以及实现和执行工具逻辑,例如根据Claude的请求运行特定函数或查询API。这让你对Claude可以使用的工具拥有完全的控制和灵活性。

具体来说,使用Claude的工具涉及以下步骤:

1. 编写工具函数功能,根据需求编写在本地可以运行的工具函数，稍后claude会通过评估用户的prompt来返回工具函数名称和参数，我们就在本地实际运行这个工具函数，其返回值可以选择返回给Claude让它进一步加工回答，也可以选择到此结束，即不用大模型加工工具函数的返回值。即所有工具都是由用户提供的，Claude没有任何内置的服务器端工具 

- 工具不一定需要是本地端函数，可以在任何需要模型返回遵循提供的模式的JSON输出时使用工具，我们只是要Claude决定使用工具时，构建格式正确的工具使用请求，即Json结构的入参，例如情感分析打分。

2. 向Claude提供工具和用户提示: (发送API请求)
    - 按照特定格式定义你希望Claude可以访问的工具函数集，通过放进API请求messages.create()中的`tools`参数中,每个函数描述{}中包括它们的名称、描述和JSon Schema（函数及所需的参数的JSON架构说明）来让Claude交互理解，tools中的要包含的键值为：
      - `name`: 工具函数名称。必须符合正则表达式 ^[a-zA-Z0-9_-]{1,64}$。即名称可以包含字母（大小写均可）、数字、下划线和短横线。名称的长度必须至少为 1 个字符，最多为 64 个字符。
      - `description`: 详细的纯文本描述,说明工具的功能、使用场景和行为。
      - `input_schema`: 一个JSON Schema对象,定义工具所需的参数。
   - 最后,我们将工具函数定义放进API请求messages.create()中的`tools`参数中发送给Claude。 

3. Claude使用工具: (解析API响应)
   - Claude评估用户prompt,并决定是否有任何可用的工具函数可以帮助回答用户的查询或完成任务。如果有,它还会决定使用哪个工具函数以及使用什么输入。`tool_choice`参数可以对Claude是否使用以及使用哪个工具函数进行选择。
   - API响应将有一个stop_reason为tool_use,表示Claude想要使用外部工具。
   - 如果决定使用工具，Claude会构建格式正确的工具使用请求。响应中会包含一个或多个`tool_use`内容块，每个块包含：
     a) `id`：该特定工具使用块的唯一标识符
     b) `name`：正在使用的工具名称
     c) `input`：传递给工具的输入对象，符合工具的`input_schema`

4. 提取工具输入、运行代码并返回结果: (发送API请求)
   - 在本地（客户端）,你应该从Claude的工具函数使用请求中提取工具名称和输入。
   - 在本地（客户端）运行实际的工具代码，执行与该工具名称对应的实际工具，传入工具的入参。
   - 使用包含`tool_result`内容块的新`user`消息结果返回给Claude，继续对话，该块包含：
     a) `tool_use_id`：这是结果对应的工具使用请求的`id`
     b) `content`：工具的结果，可以是字符串或嵌套内容块列表
     c) `is_error`（可选）：如果工具执行导致错误，设置为`true`

5. Claude使用工具结果来制定响应: (解析API响应)
   - 在收到tool_result后,Claude将使用该信息来制定其对原始用户提示的最终响应。

步骤(4)和(5)是可选的 - 对于某些工作流程,Claude使用工具就是你需要的全部信息,你可能不需要将工具结果返回给Claude。

实现一个数学计算助手

Claude 不擅长做复杂的数学运算，所以让我们通过提供对计算器工具的访问来增强 Claude 的能力。大型语言模型在数学运算方面苦苦挣扎，比如这个案例：

In [12]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

# A relatively simple math problem
response_no_tools = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content":"计算1984135 和 9343116相乘的结果，只回答结果。"}],
    max_tokens=400
)
print(response_no_tools.content[0].text)

18,576,948,360,060


In [6]:
print(1984135 * 9343116)

18538003464660


可以看到回答的并不对，我们开始编写负载着计算器功能的大模型吧！

1. 编写工具函数功能

对于我们想要编写一个实际的计算器来说，我们就在工具函数中定义好加减乘除，并思考函数的入参，即有什么原材料，背景输入给calculator函数以及Claude方不方便从用户的prompt中提取出我们需要的入参。在本calculator函数中，我们的入参有3个：

In [2]:
def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

让我们测试一下我们的函数并确保它有效。

In [4]:
print(calculator("add", 10, 3))
print(calculator("divide", 200, 25))

13
8.0


2. 向Claude提供工具和用户提示: (发送API请求)
    上面我们只是定义了自己工具箱的可以实际运行的函数，但是这离让claude理解学会还有一段距离。就像函数的作用一样，我们不需要让使用者知道内部的具体逻辑，只需要像函数注释一样知道函数说明，入参和返回值就可以。于是我们最终需要传进Claude API请求的是一段JSON格式的函数详细的说明。
    - 按照固定的官方格式定义你希望Claude可以访问的工具函数集，通过放进API请求messages.create()中的`tools`参数中,包括它们的名称、描述和JSon Schema（所需的参数的JSON架构说明）来让Claude交互理解，tools中的要包含的键值为：
      - `name`: 工具函数名称。必须符合正则表达式 ^[a-zA-Z0-9_-]{1,64}$。
      - `description`: 详细的纯文本描述,说明工具的功能、使用场景和行为。
      - `input_schema`: 一个JSON Schema对象,定义工具所需的参数。
    - 最后,我们将工具函数定义放进API请求messages.create()中的`tools`参数中发送给Claude。 
            
        假设我们定义一个发送邮件的外部工具函数，需要有接收方邮件地址，标题，邮件内容三个参数
        to which 是一个字符串，是必需的
        subject 是一个字符串，是必需的
        body 是一个字符串，是必需的
            
      ```json
        {
          "name": "send_email",
          "description": "Sends an email to the specified recipient with the given subject and body.",
          "input_schema": {
            "type": "object",
            "properties": {
              "to": {
                "type": "string",
                "description": "The email address of the recipient"
              },
              "subject": {
                "type": "string",
                "description": "The subject line of the email"
              },
              "body": {
                "type": "string",
                "description": "The content of the email message"
              }
            },
            "required": ["to", "subject", "body"]
          }
        }
     ```
     一个搜索产品的外部工具函数说明，加入的工具函数可以按名称或关键字搜索产品并筛选低于特定价格点的结果。

  ```json
    {
  "name": "search_product",
  "description": "Search for a product by name or keyword and return its current price and availability.",
  "input_schema": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "The product name or search keyword, e.g. 'iPhone 13 Pro' or 'wireless headphones'"
      },
      "category": {
        "type": "string",
        "enum": ["electronics", "clothing", "home", "toys", "sports"],
        "description": "The product category to narrow down the search results"
      },
      "max_price": {
        "type": "number",
        "description": "The maximum price of the product, used to filter the search results"
      }
    },
    "required": ["query"]
  }
}
 ```
  

In [5]:
calculator_tool = {
    "name": "calculator",
    "description": "A simple calculator that performs basic arithmetic operations.",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform."
            },
            "operand1": {
                "type": "number",
                "description": "The first operand."
            },
            "operand2": {
                "type": "number",
                "description": "The second operand."
            }
        },
        "required": ["operation", "operand1", "operand2"]
    }
}

 
- 最后，将工具函数定义放进API请求messages.create()中的`tools`参数中发送给Claude。 

In [13]:
response_with_tools = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content": "Multiply 1984135 by 9343116. Only respond with the result"}],
    max_tokens=300,
    # 给Claude装上小工具箱
    tools=[calculator_tool]
)

接下来，我们来看看 Claude 给我们的回复：

In [14]:
response_with_tools   #当Claude决定调用tools时候的响应消息对象

Message(id='msg_01GyUh8xHgwjABbV4J5KzbA7', content=[ToolUseBlock(id='toolu_01SAd6UacTU56opnFSbVJDUj', input={'operand1': 1984135, 'operand2': 9343116, 'operation': 'multiply'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=420, output_tokens=93))

In [15]:
response_no_tools    #当Claude不调用tools正常回复时候的响应消息对象

Message(id='msg_013tweRPSGWeTUxkLc98g3mJ', content=[TextBlock(text='18,576,948,360,060', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=32, output_tokens=13))

响应看起来与平时略有不同！具体来说，我们现在得到的不是普通的 Message，而是 ToolsMessage。检查response.stop_reason，看到 Claude 停止是因为它决定是时候使用一个工具了

下一步是简单地获取 Claude 提供给我们的工具名称和输入，并使用它们来实际调用我们之前编写的 calculator 函数。然后我们就会有最终的答案！

In [17]:
tool_name = response_with_tools.content[0].name
tool_inputs = response_with_tools.content[0].input

print("Claude 想要使用的函数名称是:", tool_name)
operation = tool_inputs["operation"]
operand1 = tool_inputs["operand1"]
operand2 = tool_inputs["operand2"]

result = calculator(operation, operand1, operand2)
print("Claude的结果是", result)
print("正确的结果是", 1984135 * 9343116)

Claude 想要使用的函数名称是: calculator
Claude的结果是 18538003464660
正确的结果是 18538003464660


通过System Prompt优化调用选择：

Claude 默认是非常渴望使用它的工具的，所以我们要根据需要矫正它的行为：

In [19]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{"role": "user", "content":"祖母绿是什么颜色"}],
    max_tokens=400,
    tools=[calculator_tool]
)
response

Message(id='msg_01NJsgvgfB8woQiETBwtkbjU', content=[ToolUseBlock(id='toolu_01JqwFZsp8GADDJRUGpgRpqz', input={'operand1': 0, 'operand2': 0, 'operation': 'add'}, name='calculator', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=414, output_tokens=89))

| 类别                                          | 参数名称 | 类型   | 描述                                                                 |
|---------------------------------------------|----------|--------|--------------------------------------------------------------------|
| **必需参数**                                  |          |        |                                                                    |
|                                             | model    | string | 指定要使用的 Claude 模型版本。                                     |
|                                             | messages  | array  | 输入对话历史和当前用户输入的消息列表，每个消息包括角色和内容。      |
|                                             | max_tokens | integer | 回答停止前的最大令牌数。模型可能会在达到此最大值之前停止。         |
| **可选参数**                                  |          |        |                                                                    |
| 生成控制                                      | temperature | float | 注入响应的随机性的程度。默认为 1.0，范围从 0.0 到 1.0。           |
|                                             | top_k    | integer | 仅从每个后续令牌的前 K 个选项中抽样。用于删除低概率响应的"长尾"。   |
|                                             | top_p    | float  | 使用核心抽样。在核心抽样中，计算所有选项的累积分布，并在达到特定概率后切断。 |
| **系统提示**                                  | **system** | **string** | **系统提示。用于提供上下文和指令。**                                 |
| **停止序列**                                  | **stop_sequences** | **array** | **自定义文本序列，会使模型停止生成。如果模型遇到这些序列之一，响应将提前结束。** |
| 工具调用                                      | tool_choice | string | 模型如何使用提供的工具。可以选择特定工具、任何可用工具，或由模型自行决定。 |
|                                             | tools    | array  | 定义模型可能使用的工具。每个工具定义都包括名称和输入模式。         |
| 流式传输参数                                  | stream   | boolean | 是否使用服务器发送事件增量流式传输响应。                           |
| 其他参数                                      | metadata | object | 描述请求的元数据。                                                  |
|                                             | anthropic-beta | string | 指定你想使用的 beta 版本。多个版本使用逗号分隔，不带空格。         |
|                                             | anthropic-version | string | 指定使用的 Anthropic API 的版本。                                  |
|                                             | x-api-key | string | 用于认证的独特 API 密钥。通过控制台获取。                           |

加入system参数矫正

In [21]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    system="You have access to tools, but only use them when necessary.  If a tool is not required, respond as normal",
    messages=[{"role": "user", "content":"祖母绿是什么颜色"}],
    max_tokens=400,
    tools=[calculator_tool]
)
response

Message(id='msg_01H4dLqR4mwaRaLAUKeU25HZ', content=[TextBlock(text='祖母绿是一种绿色宝石。它的颜色介于翡翠绿和孔雀绿之间,是一种非常富有典雅气质的绿色。祖母绿的颜色通常呈现偏蓝绿的翠绿色调,是很多珠宝首饰中常见的一种名贵宝石。这种碧绿色调给人一种清新优雅、高贵典雅的感受。', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=440, output_tokens=139))

一个完整的代码（步骤1~3）：

- 提供工具函数描述和用户提示

- 请求API让Claude决定是否使用工具

- 提取响应API提取参数并在本地运行代码
    


In [9]:
def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")


calculator_tool = {
    "name": "calculator",
    "description": "A simple calculator that performs basic arithmetic operations.",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The arithmetic operation to perform.",
            },
            "operand1": {"type": "number", "description": "The first operand."},
            "operand2": {"type": "number", "description": "The second operand."},
        },
        "required": ["operation", "operand1", "operand2"],
    },
}


def prompt_claude(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
        messages=messages,
        max_tokens=500,
        tools=[calculator_tool],
    )

    if response.stop_reason == "tool_use":
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input

        if tool_name == "calculator":
            print("Claude 想要调用计算器工具")
            operation = tool_input["operation"]
            operand1 = tool_input["operand1"]
            operand2 = tool_input["operand2"]

            try:
                result = calculator(operation, operand1, operand2)
                print("计算出的结果是:", result)
            except ValueError as e:
                print(f"Error: {str(e)}")

    elif response.stop_reason == "end_turn":
        print("Claude 没有调用工具")
        print("Claude 的回答:")
        print(response.content[0].text)

In [29]:
prompt_claude("我有23只羊，但是丢了两只，我还有多少只羊")

Claude 想要调用计算器工具
计算出的结果是: 21


In [11]:
prompt_claude("写一首关于秋天的古诗")

Claude 没有调用工具
Claude 的回答:
好的,我这里为您写一首关于秋天的古诗:

秋风送爽惠芳菲,
金菊吐英蓼溪流。
高柳摇阴扶风柳,
斜阳下照红叶飞。

寒鸦唱罢夕阳暮,
野客归来倦鸟栖。
篱边斜下露华凝,
古庙钟声念夕颜。

枫叶满堤秋色浓,
好风凭楼独倚东。
青山碧水空寂寥,
寂寥中有谁知逸兴?

这首诗描绘了秋天的景致,从金菊吐英到枫叶满堤,细致描摹了秋天的美好。其中还点缀了人情味,描写了归鸟栖息和独倚东楼的惬意。希望这首诗能让您感受到秋天的魅力。


接入维基百科搜索功能，构建完整的工作流

在之前的计算机的tool use流程里
步骤1~3的流程是：

提供Claude工具和用户提示
Claude决定使用工具
提取工具输入并运行代码
这个流程在Claude决定使用工具后就结束了。开发者获取到Claude要使用的工具名称和输入后，在本地执行相应的代码，但不会将结果反馈给Claude。

步骤1~5的完整流程是：

提供Claude工具和用户提示
Claude决定使用工具
提取工具输入、运行代码并返回结果
Claude使用工具结果来制定响应
这个流程多了将工具执行结果返回给Claude的步骤。Claude会根据工具返回的结果，进一步处理并给出最终回答。

如果您只需要知道Claude会使用哪个工具及其输入，那么步骤1-3就足够了。如果您希望Claude根据工具执行的结果提供更深入的分析或回答，那么完整的1-5步流程会更合适。


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240904170628.png)

【实战】：有维基百科帮助的写作助手

1. 定义我们的维基百科搜索功能

In [31]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=a5cc9020743a65a2f42f2ad2d87918a09357a4d6d38fce8d0323709b799bd446
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\c2\46\f4\caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [14]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0] #通常，第一个结果是与搜索词最相关的页面。
    page = wikipedia.page(first_result, auto_suggest=False) #获取与该标题对应的页面对象，然后通过 page.content 获取该页面的内容。
    return page.content

article = get_article("西游记")
print(article[:500]) 

Journey to the West (Chinese: 西遊記; pinyin: Xīyóu Jì) is a Chinese novel published in the 16th century during the Ming dynasty and attributed to Wu Cheng'en. It is regarded as one of the great Chinese novels, and has been described as arguably the most popular literary work in East Asia. It is widely known in English-speaking countries through Arthur Waley's 1942 abridged translation, Monkey.
The novel is a fictionalized account of the pilgrimage of the Chinese Buddhist monk Xuanzang, who journey


2. 编写工具定义

In [15]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

3. 为 Claude 提供工具和用户提示

In [16]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [{"role": "user", "content": "2024年的奥运会在哪里举行，有什么特别之处"}]

response = client.messages.create(
        model="claude-3-sonnet-20240229",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )

4. Claude 使用该工具（API 响应）

In [17]:
response.content

[TextBlock(text='让我们使用get_article工具来查找有关2024年奥运会的最新信息:', type='text'),
 ToolUseBlock(id='toolu_01W63d8twqQ4Sj5FkSM1KG7p', input={'search_term': '2024 Summer Olympics'}, name='get_article', type='tool_use')]

5. 提取工具输入、运行代码并返回结果（API 请求）


记得要添加消息到历史对话中


In [18]:
messages.append({"role": "assistant", "content": response.content})
messages

[{'role': 'user', 'content': '2024年的奥运会在哪里举行，有什么特别之处'},
 {'role': 'assistant',
  'content': [TextBlock(text='让我们使用get_article工具来查找有关2024年奥运会的最新信息:', type='text'),
   ToolUseBlock(id='toolu_01W63d8twqQ4Sj5FkSM1KG7p', input={'search_term': '2024 Summer Olympics'}, name='get_article', type='tool_use')]}]

注意可能有TextBlock来反映思考链路，所以我们取倒数第一个，即response.content[-1]来进行提取

In [19]:
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input
print("工具函数的名字: ", tool_name)
print("工具函数的输入", tool_input)

工具函数的名字:  get_article
工具函数的输入 {'search_term': '2024 Summer Olympics'}


提取LLM响应的结果中的入参，在本地运行函数获取结果

In [20]:
if tool_name == "get_article":
    search_term = tool_input["search_term"]
    wiki_result = get_article(search_term)
    print(f"搜索到了 {search_term}")
    print("维基百科的原文:")
    print(wiki_result[:500])

搜索到了 2024 Summer Olympics
维基百科的原文:
The 2024 Summer Olympics, officially the Games of the XXXIII Olympiad and branded as Paris 2024, were an international multi-sport event held from 26 July to 11 August 2024 in France, with several events started from 24 July. Paris was the host city, with events (mainly football) held in 16 additional cities spread across metropolitan France, including the sailing centre in the second-largest city of France, Marseille, on the Mediterranean Sea, as well as one subsite for surfing in Tahiti, Frenc


现在我们已经在本地执行了函数调用，我们需要将此函数调用的结果反馈给 Claude API，以便模型可以生成用户应该看到的实际响应：


1. 继续对话,发送一个新的消息,将`role`设置为`user`。 

2. 在这个新的`user`消息中,包含一个`content`块,其中包含`tool_result`类型。

3. 在`tool_result`内容块中,您需要提供以下信息:

   - `tool_use_id`: 这应该是之前工具使用请求的唯一标识符。
   - `content`: 这是工具执行的结果。它可以是一个字符串,也可以是嵌套的内容块列表。
   - `is_error`(可选): 如果工具执行出错,将其设置为`true`。

以下是一个成功工具结果的示例:

```json
{
  "role": "user",
  "content": [
    {
      "type": "tool_result",
      "tool_use_id": "toolu_01A09q90qw90lq917835lq9",
      "content": "15 degrees"
    }
  ]
}
```


In [22]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": wiki_result
        }
    ]
}

In [23]:
# Notice the long wikipedia article content!
tool_response

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_01W63d8twqQ4Sj5FkSM1KG7p',
   'content': 'The 2024 Summer Olympics, officially the Games of the XXXIII Olympiad and branded as Paris 2024, were an international multi-sport event held from 26 July to 11 August 2024 in France, with several events started from 24 July. Paris was the host city, with events (mainly football) held in 16 additional cities spread across metropolitan France, including the sailing centre in the second-largest city of France, Marseille, on the Mediterranean Sea, as well as one subsite for surfing in Tahiti, French Polynesia.\nParis was awarded the Games at the 131st IOC Session in Lima, Peru, on 13 September 2017. After multiple withdrawals that left only Paris and Los Angeles in contention, the International Olympic Committee (IOC) approved a process to concurrently award the 2024 and 2028 Summer Olympics to the two remaining candidate cities; both bids of the bids were praised for t

In [24]:
messages.append(tool_response)
print(messages)

[{'role': 'user', 'content': '2024年的奥运会在哪里举行，有什么特别之处'}, {'role': 'assistant', 'content': [TextBlock(text='让我们使用get_article工具来查找有关2024年奥运会的最新信息:', type='text'), ToolUseBlock(id='toolu_01W63d8twqQ4Sj5FkSM1KG7p', input={'search_term': '2024 Summer Olympics'}, name='get_article', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01W63d8twqQ4Sj5FkSM1KG7p', 'content': 'The 2024 Summer Olympics, officially the Games of the XXXIII Olympiad and branded as Paris 2024, were an international multi-sport event held from 26 July to 11 August 2024 in France, with several events started from 24 July. Paris was the host city, with events (mainly football) held in 16 additional cities spread across metropolitan France, including the sailing centre in the second-largest city of France, Marseille, on the Mediterranean Sea, as well as one subsite for surfing in Tahiti, French Polynesia.\nParis was awarded the Games at the 131st IOC Session in Lima, Peru, on 13 S

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240904171649.png)

准备好原材料，就可以请求API来继续处理并生成对原始用户提示的响应。

In [25]:
follow_up_response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)
print(follow_up_response.content[0].text)

根据获取的信息,我可以为您回答关于2024年奥运会的问题:

2024年奥运会将在法国巴黎举行。这将是巴黎第三次举办夏季奥运会,此前分别在1900年和1924年举办过。这届奥运会有几个特别之处:

1. 开幕式首次在传统体育场外举行。运动员们将乘船沿塞纳河游行,沿途经过巴黎的著名地标。

2. 这将是首届在参赛运动员性别比例上实现完全平等的奥运会,男女运动员人数相等。

3. 新增了霹雳舞(breaking)作为正式比赛项目。

4. 采取了多项环保措施,如奥运村使用地热自然冷却系统而非空调,食品供应增加植物性饮食比例等。

5. 安全措施空前严格,多国警察和军队参与安保工作。

6. 这是自1984年萨拉热窝冬奥会以来,首次开闭幕式在不同场地举行(开幕式在塞纳河,闭幕式在法兰西体育场)。

7. 奥运会标志和吉祥物设计都体现了法国文化元素。

这届奥运会将展现巴黎的魅力,同时也面临着诸如安全、环保等方面的挑战。组委会正在努力平衡各方面因素,力求呈现一届精彩、安全、可持续的奥运盛会。


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240904171856.png)

## tool choice

| 类别                                          | 参数名称 | 类型   | 描述                                                                 |
|---------------------------------------------|----------|--------|--------------------------------------------------------------------|
| **必需参数**                                  |          |        |                                                                    |
|                                             | model    | string | 指定要使用的 Claude 模型版本。                                     |
|                                             | messages  | array  | 输入对话历史和当前用户输入的消息列表，每个消息包括角色和内容。      |
|                                             | max_tokens | integer | 回答停止前的最大令牌数。模型可能会在达到此最大值之前停止。         |
| **可选参数**                                  |          |        |                                                                    |
| 生成控制                                      | temperature | float | 注入响应的随机性的程度。默认为 1.0，范围从 0.0 到 1.0。           |
|                                             | top_k    | integer | 仅从每个后续令牌的前 K 个选项中抽样。用于删除低概率响应的"长尾"。   |
|                                             | top_p    | float  | 使用核心抽样。在核心抽样中，计算所有选项的累积分布，并在达到特定概率后切断。 |
| **系统提示**                                  | **system** | **string** | **系统提示。用于提供上下文和指令。**                                 |
| **停止序列**                                  | **stop_sequences** | **array** | **自定义文本序列，会使模型停止生成。如果模型遇到这些序列之一，响应将提前结束。** |
| 工具调用                                      | tool_choice | string | 模型如何使用提供的工具。可以选择特定工具、任何可用工具，或由模型自行决定。 |
|                                             | tools    | array  | 定义模型可能使用的工具。每个工具定义都包括名称和输入模式。         |
| 流式传输参数                                  | stream   | boolean | 是否使用服务器发送事件增量流式传输响应。                           |
| 其他参数                                      | metadata | object | 描述请求的元数据。                                                  |
|                                             | anthropic-beta | string | 指定你想使用的 beta 版本。多个版本使用逗号分隔，不带空格。         |
|                                             | anthropic-version | string | 指定使用的 Anthropic API 的版本。                                  |
|                                             | x-api-key | string | 用于认证的独特 API 密钥。通过控制台获取。                           |


`tool_choice` 参数用于控制模型如何使用提供的工具。它有三个可能的选项：

1. `auto`：允许 Claude 自行决定是否调用任何提供的工具。如果根本不使用 tool_choice 参数，则这是使用工具时的默认行为。

2. `tool`：强制 Claude 始终使用特定工具。

3. `any`：告诉 Claude 必须使用提供的工具之一，但不强制使用特定工具。

使用 `tool_choice` 参数的方法如下：

```json
tool_choice = {"type": "tool", "name": "get_weather"}#这将强制 Claude 使用名为 "get_weather" 的工具。
tool_choice={"type": "auto"}
tool_choice={"type": "any"}
```


使用 `tool_choice` 参数时需要注意以下几点：

1. 当 `tool_choice` 设置为 `any` 或 `tool` 时，助手消息会被预填充以强制使用工具。这意味着模型不会在 `tool_use` 内容块之前生成链式思考的 `text` 内容块，即使明确要求这样做。

2. 如果您希望保持链式思考（特别是使用 Opus 模型）的同时要求模型使用特定工具，可以将 `tool_choice` 设置为默认值 `{"type": "auto"}`，并在 `user` 消息中添加明确的指示。例如：

```
What's the weather like in London? Use the get_weather tool in your response.
```

3. 使用 `tool_choice` 可以在某些情况下强制 Claude 使用特定工具来回答用户的问题，即使 Claude 认为它可以在不使用工具的情况下提供答案。

总的来说，`tool_choice` 参数提供了一种灵活的方式来控制 Claude 如何使用工具。它可以用来确保特定工具的使用，或者让 Claude 在多个工具中自由选择。根据您的具体需求和用例，合理使用这个参数可以优化 Claude 的响应。

让我们详细看看每个选项。我们首先导入 Anthropic SDK：

In [4]:
from anthropic import Anthropic
client = Anthropic()

Auto

将 tool_choice 设置为 auto 可允许模型自动决定是否使用工具。如果根本不使用 tool_choice 参数，则这是使用工具时的默认行为。

让我们从定义一个名为 web_search 的工具开始。请注意，为了简单起见，我们实际上并没有在这里搜索 Web。

In [1]:
def web_search(topic):
    print(f"进入web搜索流程搜索： {topic}")

web_search_tool = {
    "name": "web_search",
    "description": "A tool to retrieve up to date information on a given topic by searching the web",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The topic to search the web for"
            },
        },
        "required": ["topic"]
    }
}

In [6]:
from datetime import date

def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": user_query}]

    system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.  
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """
    system_prompt=f"""用你现有的知识尽可能多地回答问题。
        只有在你无法自信地回答时，才进行网络搜索。
        今天的日期是 {date.today().strftime("%Y年%m月%d日")}
        如果你认为用户的问题涉及尚未发生的未来事件，请使用搜索工具。
    """
    response = client.messages.create(
        system=system_prompt,
        model="claude-3-sonnet-20240229",
        messages=messages,
        max_tokens=1000,
        tool_choice={"type": "auto"},
        tools=[web_search_tool]
    )
    
    last_content_block = response.content[-1]
    if last_content_block.type == "text":
        print("Claude 没有调用工具")
        print(f"Claude: {last_content_block.text}")
    elif last_content_block.type == "tool_use":
        print("Claude 想要调用工具")
        print(last_content_block)
        
    print("Claude 的响应")
    print(response)

In [7]:
chat_with_web_search("天空是什么颜色的")

Claude 没有调用工具
Claude: 天空的颜色主要由太阳光线与大气层中的气体分子相互作用所致。

白天,当阳光穿过大气层时,短波长的蓝色和紫外线光被较多地分散和吸收,而长波长的红光和其他可见光则较少被影响。因此,我们看到的天空呈现出蓝色。

在日出和日落时分,由于阳光通过更长的路径穿过大气层,更多的蓝色被过滤掉,而橙红色成分占据主导,因此天空会呈现出绚丽的红色或橙色。

此外,云、污染物、灰尘等也会影响天空的色彩。总的来说,天空的颜色是由光与大气成分相互作用的lumin物理现象造成的。这是自然界一种常见而独特的光学奇观。


In [8]:
chat_with_web_search("天空为什么是蓝色的")

Claude 没有调用工具
Claude: 天空呈现蓝色的主要原因是由于日光经过大气层时发生的瑞利散射造成的。具体来说:

1. 阳光是由各种波长的电磁波组成的,其中可见光波长范围对人眼可见。

2. 当阳光射入大气层时,光线会与大气分子(主要是氮和氧)发生相互作用。

3. 瑞利散射理论指出,较短波长的蓝紫光比较长波长的红橙光更容易被大气分子散射。

4. 因此,来自太阳的蓝紫光会被大气层中的气体分子优先散射开来,使天空呈现出蓝色的颜色。

5. 而较长波长的红橙光则较少被散射,可以直接到达我们的眼睛,所以我们看到的日出日落时分呈现红橙色。

6. 白天,我们所看到的天空蓝色是大量散射的短波蓝光反射到眼睛的结果。

总的来说,天空蓝色是日光在大气层中发生选择性瑞利散射的自然现象,涉及到光的波长、大气组分和物理学规律等多方面的原因。


In [9]:
chat_with_web_search("今天有什么值得关注的新闻")

Claude 想要调用工具
ToolUseBlock(id='toolu_012wxcSADxT51T1fcoFU3L88', input={'topic': '今天的新闻头条'}, name='web_search', type='tool_use')


In [10]:
chat_with_web_search("巴黎奥运会的奖牌情况")

Claude 想要调用工具
ToolUseBlock(id='toolu_011ez9u69cdSEe22XmjqVF13', input={'topic': '2024 paris olympics medal predictions'}, name='web_search', type='tool_use')


In [16]:
chat_with_web_search("2020年奥运会的奖牌情况")

Claude 没有调用工具
Claude: 2020年东京奥运会是一个非常成功和令人难忘的体育盛事。让我概括一下主要奖牌情况:

金牌总数最多的前三名国家/地区是:

1. 美国 - 113金
2. 中国 - 88金 
3. 日本 - 58金

中国在本届奥运会上共获得88枚金牌,38枚银牌和53枚铜牌,奖牌总数为179枚,位列奖牌种类榜第二位。

在奥运会上,中国军团在羽毛球、射击、跆拳道、艺术体操、蹦床等项目上表现出色,包揽多枚金牌。

值得一提的是,中国选手在女子全能操夺冠,在体操项目上延续了好成绩。总的来说,虽然金牌总数略少于上届里约奥运会,但中国代表团的整体表现依然处于世界前列。

我概括了一些关于2020年东京奥运会中国奖牌情况的主要信息。如果你还有其他问题,欢迎继续询问。


强制 Claude 始终使用特定工具-tool

1. `auto`：允许 Claude 自行决定是否调用任何提供的工具。如果根本不使用 tool_choice 参数，则这是使用工具时的默认行为。

2. `tool`：强制 Claude 始终使用特定工具。

3. `any`：告诉 Claude 必须使用提供的工具之一，但不强制使用特定工具。

【实战】文本情感分析的JSON格式输出

如今，大多数人每天都在网上发布或阅读意见。其中包括对产品或品牌的大量意见，因此跟踪有关其业务的评论，关注社交媒体上的情绪也是一个巨大的需求。

在我的工具箱里有两个工具，分别是给情感分数打分的工具函数（目标工具）和计算器（扰乱工具）。

现在的需求是使用 Claude 对给定文本执行情绪分析，并以结构化 JSON 格式返回情绪分数。

目标如：
```json
  {
  "positive_score": 0.8,
  "negative_score": 0.0,
  "neutral_score": 0.2
}
```

此处涉及两个知识点，一个就是强制使用‘tool’，另一个是当需要以结构化 JSON 格式返回时除了prompt提示“请用json格式返回”，或者预填写Claude的回答



老办法，用prompt限制：

In [26]:
def analyze_tweet_sentiment_prompt(query):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        system=
            """
                分析以下文本的情感并返回一个包含情感分数的 JSON 格式对象。
                该对象应包含以下三个键：
                - "positive_score" (float): 正面情感分数，范围从 0.0 到 1.0。
                - "negative_score" (float): 负面情感分数，范围从 0.0 到 1.0。
                - "neutral_score" (float): 中性情感分数，范围从 0.0 到 1.0。
                示例：
                {
                  "positive_score": 0.3,
                  "negative_score": 0.1,
                  "neutral_score": 0.6
                }
            """,
    
        max_tokens=500,
        messages=[{"role": "user", "content": query}],
    )
    print(response.content[0].text)

In [27]:
analyze_tweet_sentiment_prompt("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")

对于您的文本分析,结果如下:

{
  "positive_score": 0.8,
  "negative_score": 0.0,
  "neutral_score": 0.2
}

这表示您的文本中包含较多的积极情感(喜欢猫),没有负面情感,中性情感占比较低。这反映了您对于养猫的积极态度和计划。

根据您的陈述,如果您现在已有1只猫,并打算再养1只,那么您最终将会拥有2只猫。所以您现在的猫只数是1只,未来将变成2只。


用预填写Claude的回应优化：

In [27]:
def analyze_tweet_sentiment_prompt(query):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        system=
            """
                分析以下文本的情感并返回一个包含情感分数的 JSON 格式对象。
                该对象应包含以下三个键：
                - "positive_score" (float): 正面情感分数，范围从 0.0 到 1.0。
                - "negative_score" (float): 负面情感分数，范围从 0.0 到 1.0。
                - "neutral_score" (float): 中性情感分数，范围从 0.0 到 1.0。
                示例：
                {
                  "positive_score": 0.3,
                  "negative_score": 0.1,
                  "neutral_score": 0.6
                }
            """,
    
        max_tokens=500,
        messages=[{"role": "user", "content": query},{"role": "assistant", "content": "{"}],
    )
    print(response.content[0].text)
    


In [28]:
analyze_tweet_sentiment_prompt("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")


  "positive_score": 0.7,
  "negative_score": 0.0,
  "neutral_score": 0.3
}

这个回答表示:
- 正面情感分数较高(0.7),表示你对于养猫这件事感到正面和积极。
- 负面情感分数很低(0.0),说明你没有什么负面情绪。
- 中性情感分数较低(0.3),表示你对此事还没有完全确定和确切的感觉。

总的来说,你对于养猫这件事持有积极正面的态度,但可能还需要一些时间来完全确定和决定。你目前有1只猫,想再养一只,所以如果你真的再养1只,你最终会拥有2只猫。


继续利用以往学习的知识进行优化，添加stop_sequences：

In [29]:
def analyze_tweet_sentiment_prompt(query):
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        system=
            """
                分析以下文本的情感并返回一个包含情感分数的 JSON 格式对象，只返回JSON对象就可以。
                该对象应包含以下三个键：
                - "positive_score" (float): 正面情感分数，范围从 0.0 到 1.0。
                - "negative_score" (float): 负面情感分数，范围从 0.0 到 1.0。
                - "neutral_score" (float): 中性情感分数，范围从 0.0 到 1.0。
                示例：
                {
                  "positive_score": 0.3,
                  "negative_score": 0.1,
                  "neutral_score": 0.6
                }
            """,
    
        max_tokens=100,
        messages=[{"role": "user", "content": query},{"role": "assistant", "content": "{"}],
        stop_sequences=["}"]
    )
    print(response.content[0].text)

In [30]:
analyze_tweet_sentiment_prompt("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")


  "positive_score": 0.8,
  "negative_score": 0.0,
  "neutral_score": 0.2
}


开始用本节课学的知识完成它！

通过函数调用来达到得到大模型返回Json格式的效果，重点是大模型会以规范的格式输出入参。

并为了展现tool_choice参数中tool的用法，我们在工具箱中添加一个扰乱函数。

In [30]:
tools = [
    # 打印情感分数的函数。
    # 参数:
    # - positive_score (float): 正面情感分数，范围从 0.0 到 1.0。
    # - negative_score (float): 负面情感分数，范围从 0.0 到 1.0。
    # - neutral_score (float): 中性情感分数，范围从 0.0 到 1.0。
    {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given tweet or piece of text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {"type": "number", "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
                "negative_score": {"type": "number", "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
                "neutral_score": {"type": "number", "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
        }
    },
    {
        "name": "calculator",
        "description": "Adds two number",
        "input_schema": {
            "type": "object",
            "properties": {
                "num1": {"type": "number", "description": "first number to add"},
                "num2": {"type": "number", "description": "second number to add"},
            },
            "required": ["num1", "num2"]
        }
    }
]

"反面例子"——原本的结果：

In [18]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "auto"},
        messages=[{"role": "user", "content": query}]
    )
    print(response)

In [19]:
analyze_tweet_sentiment("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")

Message(id='msg_01CJ3cbo72vpjfk6UhhD91gQ', content=[TextBlock(text='好的,让我们使用计算器工具来算一下:', type='text'), ToolUseBlock(id='toolu_01N9yJt9vJAHZxYDYwmUbyZu', input={'num1': 1, 'num2': 1}, name='calculator', type='tool_use')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=454, output_tokens=90))


改变tool_choice

In [ ]:
tool_choice={"type": "auto"}
tool_choice={"type": "tool", "name": "print_sentiment_scores"}

In [31]:
import json
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "tool", "name": "print_sentiment_scores"},
        messages=[{"role": "user", "content": query}]
    )
    print(response)
    json_sentiment = None
    for content in response.content:
        if content.type == "tool_use" and content.name == "print_sentiment_scores":
            json_sentiment = content.input
            break
    
    if json_sentiment:
        print("情感分析 (JSON):")
        print(json.dumps(json_sentiment, indent=2))

In [32]:
analyze_tweet_sentiment("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")

Message(id='msg_01NtQ9knRCYX4Esr5yC5r3AM', content=[ToolUseBlock(id='toolu_01SrfSEzF78kDfKBTmzZKy7A', input={'positive_score': 0.2, 'negative_score': 0.1, 'neutral_score': 0.7}, name='print_sentiment_scores', type='tool_use')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=551, output_tokens=79))
情感分析 (JSON):
{
  "positive_score": 0.2,
  "negative_score": 0.1,
  "neutral_score": 0.7
}


any

"any"是告诉Claude它必须使用提供的工具之一，但不强制使用特定的工具。当tool_choice设置为"any"和"tools"时，Claude会被预先填充助手消息以强制使用工具。这意味着模型不会在tool_use内容块之前输出链式思考（chain-of-thought）的text内容块，即使被明确要求这样做。

其实这是因为在使用对于tool_choice中的"tool"选项和"any"选项时，系统会自动包含一个特殊的系统提示来启用工具使用。不同模型的token数量略有不同。例如：
Claude 3.5 Sonnet: 261 tokens
Claude 3 Opus: 281 tokens
Claude 3 Sonnet: 235 tokens
Claude 3 Haiku: 340 tokens 2

如果在构建agent等应用时希望在请求使用特定工具的同时保持链式思考（特别是对于Opus模型），您可以使用tool_choice的默认值{"type": "auto"}

In [37]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        # tool_choice="auto" 这种写法是不对的
        tool_choice={"type": "auto"},
        messages=[{"role": "user", "content": query}]
    )
    print(response)
    json_sentiment = None
    for content in response.content:
        if content.type == "tool_use" and content.name == "print_sentiment_scores":
            json_sentiment = content.input
            break
    
    if json_sentiment:
        print("情感分析 (JSON):")
        print(json.dumps(json_sentiment, indent=2))

In [38]:
analyze_tweet_sentiment("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")

Message(id='msg_01Fs2KSccw3955rez5YvJfWQ', content=[TextBlock(text='这是一个简单的加法问题。让我们使用"calculator"工具来计算:', type='text'), ToolUseBlock(id='toolu_011eomQ5PgZ2eHG7PYP5rJZf', input={'num1': 1, 'num2': 1}, name='calculator', type='tool_use')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=453, output_tokens=98))


In [39]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system="要有思维链的过程，先思考并返回思考结果，再调用工具",
        max_tokens=4096,
        tools=tools,
        # tool_choice="any" 这种写法是不对的
        tool_choice={"type": "any"},
        messages=[{"role": "user", "content": query}]
    )
    print(response)
    json_sentiment = None
    for content in response.content:
        if content.type == "tool_use" and content.name == "print_sentiment_scores":
            json_sentiment = content.input
            break
    
    if json_sentiment:
        print("情感分析 (JSON):")
        print(json.dumps(json_sentiment, indent=2))

In [40]:
analyze_tweet_sentiment("我喜欢我的猫，我已经有了1只，现在还想养一只，我会有几只猫?")

Message(id='msg_01MPNjVovA4oRJu95YwppB7Y', content=[ToolUseBlock(id='toolu_012jQw4EDeMSLpBWxA1haJ98', input={'num1': 1, 'num2': 1}, name='calculator', type='tool_use')], model='claude-3-sonnet-20240229', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=570, output_tokens=55))




### 【实战】构建模拟处理客户关系管理(CRM)和订单管理系统聊天机器人

构建一个智能化的企业管理助手,集成了客户信息管理和订单跟踪功能。

1. 快速检索客户信息:包括姓名、邮箱、电话等关键联系方式（根据模拟的数据库信息而定）。

2. 实时查询订单状态:能够即时获取订单的处理进度、产品详情、数量和价格等信息（根据模拟的数据库信息而定）。

3. 智能对话交互:通过自然语言处理,理解用户查询意图,提供精准的信息反馈。

4. 信息整合与展示:将分散的数据整合,以清晰、结构化的方式呈现给用户。

5. 主动信息补充:在回答用户主要问题的同时,还会提供相关的额外信息,增加信息的完整性。

6. 持续服务承诺:始终保持开放态度,鼓励用户进行更深入的询问。

加入我们不做路演，那么学习它一是体验工具箱有多个工具来完成连续的调用了工具函数的对话

第 1 步：设置环境

In [1]:
import anthropic

client = anthropic.Client()
MODEL_NAME="claude-3-5-sonnet-20240620"

第 2 步：定义客户端工具函数说明JSON集

接下来，我们将定义聊天机器人将用于帮助客户的客户端工具。我们将创建三个工具：get_customer_info、get_order_details 和 cancel_order。

In [45]:
tools = [
    {
        "name": "get_customer_info",
        "description": "根据客户ID获取客户信息。返回客户的姓名、电子邮件和电话号码。",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "客户的唯一标识符。"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "get_order_details",
        "description": "根据订单ID获取特定订单的详细信息。返回订单ID、产品名称、数量、价格和订单状态。",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "订单的唯一标识符。"
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "根据提供的订单ID取消订单。如果取消成功，返回确认消息。",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "要取消的订单的唯一标识符。"
                }
            },
            "required": ["order_id"]
        }
    }
]

第 3 步：模拟工具响应：由于我们没有真实的客户数据或订单信息，因此我们将为我们的工具模拟合成响应。在实际场景中，这些功能将与我们的实际客户数据库和订单管理系统进行交互。

In [3]:
def get_customer_info(customer_id):
    # 模拟的客户数据
    customers = {
        "C1": {"name": "马冬梅", "email": "dongmei@example.com", "phone": "123-456-7890"},
        "C2": {"name": "小帅", "email": "shuai@example.com", "phone": "987-654-3210"}
    }
    return customers.get(customer_id, "客户未找到")  

def get_order_details(order_id):
    # 模拟的订单数据
    orders = {
        "O1": {"id": "O1", "product": "苹果6", "quantity": 2, "price": 19.99, "status": "已发货"},
        "O2": {"id": "O2", "product": "苹果6 Plus", "quantity": 1, "price": 49.99, "status": "处理中"}
    }
    return orders.get(order_id, "订单未找到")  

def cancel_order(order_id):
    # 模拟的订单取消
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False


步骤 4：处理工具调用并返回结果。创建一个函数来处理 Claude 进行的工具调用并返回相应的结果。

In [4]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

第 5 步：与聊天机器人交互

In [50]:
import json

def chatbot_interaction(user_message):
    print(f"\n{'='*50}\n用户消息: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    # 发送初始请求
    response = client.messages.create(
        model=MODEL_NAME,
        system="我们的工具集可以查询客户电子邮件和订单相关信息",
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\n初始响应:")
    print(f"停止原因: {response.stop_reason}")
    print(f"内容: {response.content}")

    # 处理工具调用循环
    while response.stop_reason == "tool_use":
        # 获取工具使用信息
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\n使用的工具: {tool_name}")
        print(f"工具输入:")
        print(json.dumps(tool_input, indent=2))

        # 处理工具调用
        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\n工具结果:")
        print(json.dumps(tool_result, indent=2))

        # 构建包含工具结果的新消息
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        # 发送后续请求
        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            # tool_choice={"type": "any"},
            messages=messages
        )

        print(f"\n响应:")
        print(f"停止原因: {response.stop_reason}")
        print(f"内容: {response.content}")

    # 提取最终响应
    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\n最终响应: {final_response}")

    return final_response

In [51]:
chatbot_interaction("你能告诉我客户C1的电子邮件地址吗？")
chatbot_interaction("订单O2的状态是什么？")
chatbot_interaction("请帮我取消订单O1。")
chatbot_interaction("请帮我取消订单O4。")


用户消息: 你能告诉我客户C1的电子邮件地址吗？

初始响应:
停止原因: tool_use
内容: [TextBlock(text='当然可以。为了获取客户C1的电子邮件地址，我需要使用 get_customer_info 函数。我会使用客户ID "C1" 来查询这个信息。让我为您查询一下。', type='text'), ToolUseBlock(id='toolu_01HeVdpAiZoqqdnkpmMRcSJK', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

使用的工具: get_customer_info
工具输入:
{
  "customer_id": "C1"
}

工具结果:
{
  "name": "\u9a6c\u51ac\u6885",
  "email": "dongmei@example.com",
  "phone": "123-456-7890"
}

响应:
停止原因: end_turn
内容: [TextBlock(text='根据查询结果，我可以告诉您客户C1的信息：\n\n客户C1的姓名是马冬梅，电子邮件地址是 dongmei@example.com。\n\n除此之外，如果您需要的话，我还可以提供她的电话号码。有什么其他信息您想了解的吗？', type='text')]

最终响应: 根据查询结果，我可以告诉您客户C1的信息：

客户C1的姓名是马冬梅，电子邮件地址是 dongmei@example.com。

除此之外，如果您需要的话，我还可以提供她的电话号码。有什么其他信息您想了解的吗？

用户消息: 订单O2的状态是什么？

初始响应:
停止原因: tool_use
内容: [TextBlock(text='为了获取订单O2的状态信息，我需要使用get_order_details函数来查询订单详情。我会使用您提供的订单ID "O2" 来进行查询。', type='text'), ToolUseBlock(id='toolu_016SCxtazGVSwe4oCiNnoQor', input={'order_id': 'O2'}, name='get_order_details', type='tool_use')]

使

'很抱歉,看起来取消订单O4的操作没有成功。系统返回了False,这可能意味着订单无法被取消。这种情况可能有几个原因:\n\n1. 订单可能已经在处理中或已发货,因此无法取消。\n2. 订单ID可能不正确或不存在。\n3. 可能存在系统错误。\n\n为了更好地了解情况,我建议我们查看一下这个订单的详细信息。这样我们就能知道为什么无法取消,以及订单目前的状态。您同意我查看订单详情吗？'